In [1]:
from data import preparation
import data.hierarchy as hie
from sklearn.model_selection import train_test_split

In [2]:
with open('data/wipo_d/hierarchy.txt', 'w') as f1:
    with open('data/wipo_d/wipo_d.ht') as f:
        for l in f:
            split = l.strip().split(':')
            p = split[0]
            c = split[1].split(',')
            for cc in c:
                f1.write('%s %s\n'% (p,cc))

In [3]:
data_name = "wipo_d"

In [4]:
hierarchy, parent_of, all_name, name_to_index, level = hie.reindex_hierarchy(
                '%s/hierarchy.txt' % data_name)
hie.save_hierarchy("%s/hierarchy.pickle" % data_name, hierarchy,
                   parent_of, all_name, name_to_index, level)

In [5]:
for mode in ['train', 'test']:
    for i in range(1,6):
        datas, labels = preparation.import_data('wipo_d/folds/wipo_d_fold%d.dat.%s' % (i, mode))
        new_labels = preparation.map_index_of_label('wipo_d/hierarchy.pickle', labels)
        if mode == 'train':
            train_data, validate_data, train_target, validate_target = train_test_split(
                datas, new_labels, test_size=0.1, random_state=12345)
            preparation.save_data_in_pickle('wipo_d/fold/data_%d.pickle.%s' % (i, mode), train_data, train_target)
            preparation.save_data_in_pickle('wipo_d/fold/data_%d.pickle.%s' % (i, "validate"), validate_data, validate_target)
        else:
            preparation.save_data_in_pickle('wipo_d/fold/data_%d.pickle.%s' % (i, mode), datas, new_labels)

# Real Flow 

In [6]:
from data import Dataset
from embedding import Doc2Vec
from assemble_classifier import AssembleNoLabel, AssemblePredicted
%load_ext autoreload
%autoreload 2

In [14]:
dataset_train = Dataset("wipo_d", 1, "train", state='embedding')
dataset_validate = Dataset("wipo_d", 1, "validate", state='embedding')
dataset_test = Dataset("wipo_d", 1, "test", state='embedding')

In [17]:
# doc2vec = Doc2Vec(dataset_train.number_of_classes(), size=100, epoch=500)
# doc2vec.fit(dataset_train.datas, dataset_train.labels, dataset_validate.datas, dataset_validate.labels)
# doc2vec.load_model("best_now/doc2vec.model")

In [18]:
# dataset_train.change_to_Doc2Vec(doc2vec)
# dataset_validate.change_to_Doc2Vec(doc2vec)
# dataset_test.change_to_Doc2Vec(doc2vec)

In [19]:
model = AssemblePredicted("wipo_d", dataset_train, dataset_validate, dataset_test, iteration=20000, batch_size=3000, hidden_size=[300,1500,1500,1500], target_hidden_size=[60,60,60], use_dropout=True, start_level=0)

In [20]:
model.train()

Level: 1.000 Epoch: 666/20000 Batch: 1/1 Loss: 0.051 Training Loss: 0.051 Validate F1 macro: 0.638
Level: 1.000 Epoch: 1166/20000 Batch: 1/1 Loss: 0.012 Training Loss: 0.012 Stopping F1 macro: 0.601

Level: 2.000 Epoch: 666/20000 Batch: 1/1 Loss: 0.108 Training Loss: 0.108 Validate F1 macro: 0.544
Level: 2.000 Epoch: 1166/20000 Batch: 1/1 Loss: 0.064 Training Loss: 0.064 Stopping F1 macro: 0.515

Level: 3.000 Epoch: 666/20000 Batch: 1/1 Loss: 0.625 Training Loss: 0.625 Validate F1 macro: 0.089
Level: 3.000 Epoch: 1183/20000 Batch: 1/1 Loss: 0.256 Training Loss: 0.256 Stopping F1 macro: 0.078

Level: 4.000 Epoch: 666/20000 Batch: 1/1 Loss: 0.889  Training Loss: 0.889 Validate F1 macro: 0.008
Level: 4.000 Epoch: 1332/20000 Batch: 1/1 Loss: 0.233 Training Loss: 0.233 Validate F1 macro: 0.014
Level: 4.000 Epoch: 1510/20000 Batch: 1/1 Loss: 0.185 Training Loss: 0.185 Stopping F1 macro: 0.012



In [21]:
model.tuning_threshold()

In [22]:
f1_macro, f1_micro, f1_each = model.evaluate("test")

In [23]:
print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))

F1 macro: 0.0915 F1 micro: 0.4824


In [24]:
for level, (macro, micro) in enumerate(f1_each):
    print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))

Level: 0 F1 macro: 0.6065 F1 micro: 0.7408
Level: 1 F1 macro: 0.5631 F1 micro: 0.6094
Level: 2 F1 macro: 0.1572 F1 micro: 0.3576
Level: 3 F1 macro: 0.0550 F1 micro: 0.2054
